## Customer Dataset Cleaning

In [1]:
# %pip install fuzzywuzzy
# !python -m pip install --upgrade pip
# !pip install pycountry
# %pip install python-Levenshtein
# %pip install pandas numpy matplotlib seaborn scikit-learn jupyter

In [ ]:
# Activate kernal:
# .\venv_fyp\Scripts\activate

# Step 1: Import libraries
import pandas as pd
import numpy as np
import os
from datetime import datetime, date
from fuzzywuzzy import process, fuzz
import pycountry
import re

In [3]:
# Step 2: Load dataset
# Replace 'customer_dataset.csv' with your actual file name or path
file_path = "C:/Users/user/OneDrive/Desktop/Onedrive_YuyanDipsy/OneDrive/UM Y4S1/WIA3002 FYP 1 & 2/FYP2/Data/Soapan Santun/2021 - 2025 Customer - Copy.csv"

original_customer_dataset_name = "2021 - 2025 Customer - Copy.csv"

# Read dataset
customer_df = pd.read_csv(file_path)

# Show first few rows (original raw data)
customer_df.head()

,CustomerID,Date of Birth,Gender,City,State,Country
0,CUST0001,NaN,NaN,Sungai Besi,Kuala Lumpur,Malaysia
1,CUST0001,NaN,NaN,NaN,NaN,NaN
2,CUST0002,NaN,NaN,Mutiara Damansara,Selangor,Malaysia
3,CUST0002,NaN,NaN,NaN,NaN,NaN
4,CUST0003,NaN,NaN,Shah Alam,Selangor,Malaysia


### Data Profiling

In [4]:
## Create later

### Perform Data Cleaning Pipeline - CustomerDataset

In [ ]:
# ============================================= STAGE 1: SCHEMA & COLUMN VALIDATION =============================================
# # Optional columns
def check_optional_columns(df, threshold=0.8):
    """
    Check optional columns for fill percentage and drop columns that are mostly empty.
    Returns the modified DataFrame and a friendly message.
    """
    
    optional_columns = ["date of birth", "gender"]
    dropped_columns = []

    for col in optional_columns:
        if col in df.columns:
            fill_ratio = df[col].count() / len(df) 
            if fill_ratio < threshold:
                dropped_columns.append(col)
                df.drop(columns=[col], inplace=True)  # Drop the column immediately
                # df[col].count(): This counts the number of non-missing (non-null/non-NaN) values in the current column (col).
                # len(df): This gives the total number of rows in the DataFrame.
                # fill_ratio: The division calculates the proportion of filled (non-missing) values in that column. A ratio of 1.0 means the column is entirely filled; a ratio of 0.1 means 90% of the values are missing.

    # Generate user-friendly message
    if dropped_columns:
        dropped_str = ", ".join(dropped_columns)
        message = (
            f"We noticed that very few entries were provided for {dropped_str}. "
            "These columns have been removed. "
            "Segmentation will still be performed using geographic (City, State, Country) "
            "and behavioral data (e.g., orders, purchase items, total spend)."
        )
    else:
        message = "All optional columns have enough data and are kept for analysis."
    
    return df, message

# Mandatory columns 
def check_mandatory_columns(df, threshold=0.8):
    """
    Check mandatory columns for missing values (column-wise).
    Does not drop columns — only warns user if any column is too incomplete.
    Returns the DataFrame and a message summarizing issues.
    """

    mandatory_columns = ["customerid", "city", "state"]

    missing_report = []
    warning_columns = []

    for col in mandatory_columns:
        if col in df.columns:
            fill_ratio = df[col].count() / len(df)
            missing_percent = (1 - fill_ratio) * 100

            missing_report.append(f"{col}: {missing_percent:.1f}% missing")

            # Warn if missing exceeds threshold
            if fill_ratio < threshold:
                warning_columns.append(col)
        else:
            # Handle case where column completely missing
            missing_report.append(f"{col}: column not found (100% missing)")
            warning_columns.append(col)

    # Generate friendly message
    if warning_columns:
        warning_str = ", ".join(warning_columns)
        message = (
            f"Some key fields have a high number of missing values: {warning_str}. "
            "The system will still continue cleaning and processing, "
            "but missing values will be handled automatically by our system. "
            "Please ensure your source data is as complete as possible for more accurate segmentation results.\n\n"
            "Missing Data Summary:\n" + "\n".join(missing_report)
        )
    else:
        message = (
            "All mandatory columns have sufficient data and are ready for cleaning.\n\n"
            "Missing Data Summary:\n" + "\n".join(missing_report)
        )

    return df, message

# ============================================= STAGE 2: REMOVE DUPLICATE ENTRY ROW =================================================
def remove_duplicate_entries(df):
    """Remove duplicate rows, keeping the first occurrence"""
    df = df.drop_duplicates(keep='first')
    return df

# ============================================= STAGE 3: STANDARDIZATION & NORMALIZATION =============================================

def normalize_columns_name(df):
    """Normalize column names: lowercase, strip spaces"""
    df.columns = df.columns.str.strip().str.lower()
    return df

# ===============================================================================

def standardize_customer_id(df):
    """Standardize CustomerID format"""
    if 'customerid' in df.columns:
        df['customerid'] = df['customerid'].astype(str).str.strip().str.upper()
    return df
# Might have special case of dirty data exist such as "****", "1234....", "annbwbciwbciowb"
# not sure how to handle it (Currently will say bcs we focus on small business enterprise that have use digital system, so normally customerID will not have inconsistent format issue, even the inconsistant format exist, at the end this row will not be use as when we merge we cant found that customerID)

# ===============================================================================

def standardize_dob(df):
    """Standardize Date of Birth column and convert to YYYY-MM-DD"""
    # Rename only 'date of birth' to 'dob'
    df = df.rename(columns={'date of birth': 'dob'})  
    if 'dob' in df.columns:
        def parse_date(x):
            if pd.isnull(x):
                return np.nan
            for format in ("%d/%m/%Y", "%m-%d-%y", "%Y-%m-%d", "%d-%b-%Y", "%d-%m-%Y"):    
                try:
                    return datetime.strptime(str(x), format).date() # Final format: YYYY-MM-DD | 2025-10-15
                except Exception:
                    continue
            return np.nan  # If no valid format found
        df['dob'] = df['dob'].apply(parse_date)
    return df

# %d/%m/%Y → 12/05/2000
# %m-%d-%y → 05-12-00
# %Y-%m-%d → 2000-05-12
# %d-%b-%Y → 12-May-2000
# %d-%m-%Y → 12-5-2000

# ===============================================================================

def derive_age_features(df):
    """Derive Age from DOB"""
    if 'dob' in df.columns:
        today = date.today()
        df['age'] = df['dob'].apply(
            lambda x: today.year - x.year - ((today.month, today.day) < (x.month, x.day))
            if pd.notnull(x) else np.nan
        )
    return df
# Example: ((today.month, today.day) < (x.month, x.day))
# (10,15) < (12,1) → True (birthday in Dec is after Oct 15)
# (10,15) < (10,16) → True (birthday tomorrow)
# (10,15) < (5,20) → False (birthday already passed)

# This function calculates each person’s age from their date of birth (dob) by subtracting their birth year from the current year and adjusting if their birthday hasn’t occurred yet this year.

# ===============================================================================

def derive_age_group(df):
    """Derive Age Group based on defined buckets"""
    if 'age' in df.columns:
        def categorize_age(age):
            if pd.isnull(age):
                return 'Unknown'
            if age < 18: return 'Below 18'
            elif 18 <= age <= 24: return '18-24'
            elif 25 <= age <= 34: return '25-34'
            elif 35 <= age <= 44: return '35-44'
            elif 45 <= age <= 54: return '45-54'
            elif 55 <= age <= 64: return '55-64'
            else: return 'Above 65'
        df['age_group'] = df['age'].apply(categorize_age)
    return df
# ===============================================================================

def drop_dob_after_age_derived(df):
    """Drop DOB column after deriving age and age_group"""
    if 'dob' in df.columns:
        df = df.drop(columns=['dob'])
    return df

# =================================================================================

def standardize_gender(df):
    """Clean and standardize gender values"""
    if 'gender' in df.columns:
        # Clean text (remove spaces, make lowercase)
        df['gender'] = df['gender'].astype(str).str.strip().str.lower()

        # Standardize using keyword detection
        def detect_gender(value):
            if any(word in value for word in ['m', 'male', 'man', 'boy']):
                return 'Male'
            elif any(word in value for word in ['f', 'female', 'woman', 'girl']):
                return 'Female'
            else:
                return 'Unknown'

        df['gender'] = df['gender'].apply(detect_gender)
    return df

# ==================================================================================

def standardize_location(df):
    """Standardize City, and State fields"""
    
    # Helper function: detect suspicious city names
    def is_suspicious_city(name):
        if not name or name.strip() == '':
            return True
        name = str(name).strip()
        # Too short or too long
        if len(name) < 2 or len(name) > 50:
            return True
        # Contains non-alphabetic or weird symbols
        if re.search(r'[^A-Za-z\s\'-]', name):  # letters, space, apostrophe, dash allowed
            return True
        # Repeated characters (e.g., "Ccciiiittty")
        if re.search(r'(.)\1{3,}', name):
            return True
        return False

    # --- City ---
    if 'city' in df.columns:
        df['city'] = df['city'].astype(str).str.title().str.strip()
        df['city'] = df['city'].apply(lambda x: 'Unknown' if is_suspicious_city(x) else x)

    # --- State ---
    if 'state' in df.columns:
        malaysia_states = [sub.name for sub in pycountry.subdivisions if sub.country_code == 'MY']
        df['state'] = df['state'].astype(str).str.title().str.strip()
        df['state'] = df['state'].apply(
            lambda x: process.extractOne(x, malaysia_states, scorer=fuzz.token_sort_ratio)[0] if x else 'Unknown'
        )

    return df

# ============================================= STAGE 4: MISSING VALUE HANDLING =============================================

def handle_missing_values(df):
    """
    Handle missing values using a column-based approach.
    Each column is treated independently based on its type and business logic.
    """

    # ----- 1. Customer ID -----
    if 'customerid' in df.columns:
        df = df[df['customerid'].notna()]  # Drop rows without ID

    # ----- 2. Age -----
    if 'age' in df.columns:
        missing_ratio = df['age'].isna().mean()

        if missing_ratio > 0:
            print(f"Missing ratio for age: {missing_ratio:.2%}")
            
            if 'gender' in df.columns and df['gender'].nunique() > 1:
                # Group by gender if available
                df['age'] = df.groupby('gender')['age'].transform(
                    lambda x: x.fillna(x.median())
                )
                print("Applied gender-based median imputation for age.")
            else:
                # No gender column found or only one unique gender
                print("Gender column not available or not diverse. Using overall median for imputation.")
            
            # Always fill any remaining missing values with overall median
            df['age'] = df['age'].fillna(df['age'].median())
            
            # Update derived column if needed
            df = derive_age_group(df)

    # ----- 4. Gender -----
    if 'gender' in df.columns:
        df['gender'] = df['gender'].fillna(df['gender'].mode()[0])
# CHECK THE REST (SUSU, HOPE CAN SOLVE BY TOMORROW)
    # ----- 5. City -----
    if 'city' in df.columns and 'state' in df.columns:
        # Precompute the most common city per state once
        city_mode_by_state = (
            df[df['city'] != 'Unknown']
            .groupby('state')['city']
            .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'Unknown')
            .to_dict()
        )

        # Replace 'Unknown' with that state's most common city if available
        df['city'] = df.apply(
            lambda r: city_mode_by_state.get(r['state'], 'Unknown')
            if r['city'] == 'Unknown' and pd.notna(r['state'])
            else r['city'],
            axis=1
        )

    # --- State ---
    if 'state' in df.columns:
        df['state'] = df['state'].fillna(df['state'].mode()[0])
    return df

# ============================================= STAGE 5: OUTLIER DETECTION =============================================

def detect_outliers(df):
    """Detect outliers in Age"""
    if 'age' in df.columns:
        # Convert to numeric just in case
        df['age'] = pd.to_numeric(df['age'], errors='coerce')

        # --- 1️⃣ Detect outliers using IQR method ---
        Q1 = df['age'].quantile(0.25)
        Q3 = df['age'].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Mark extreme outliers as NaN
        df.loc[(df['age'] < lower_bound) | (df['age'] > upper_bound), 'age'] = np.nan

        print(f"Outlier detection complete. Replaced ages outside [{lower_bound:.1f}, {upper_bound:.1f}] with NaN.")

        # --- 2️⃣ Handle missing/outlier ages ---
        missing_ratio = df['age'].isna().mean()
        if missing_ratio > 0:
            print(f"Missing ratio for age: {missing_ratio:.2%}")
            
            if 'gender' in df.columns and df['gender'].nunique() > 1:
                # Group by gender if available
                df['age'] = df.groupby('gender')['age'].transform(
                    lambda x: x.fillna(x.median())
                )
                print("Applied gender-based median imputation for age.")
            else:
                # No gender column found or only one unique gender
                df['age'] = df['age'].fillna(df['age'].median())
                print("Applied overall median imputation for age.")
            
        # --- 3️⃣ Update derived column ---
        df = derive_age_group(df)
        return df

# ============================================= STAGE 6: DEDUPLICATION =============================================
# This function removes duplicate CustomerIDs and keeps the row with the most non-missing data to preserve the most complete customer record.
def deduplicate_customers(df):
    """Handle duplicate customer records intelligently"""
    if 'customerid' not in df.columns:
        return df  # Skip if no customerid column
    
    merged_records = []

    for cust_id, group in df.groupby('customerid'):
        merged_row = {'customerid': cust_id}
        for col in df.columns:
            if col == 'customerid':
                continue
            
            # Get all unique non-null values for this customer
            values = group[col].dropna().unique()
            
            if len(values) == 0:
                merged_row[col] = pd.NA
            elif len(values) == 1:
                merged_row[col] = values[0]
            else:
                # Multiple conflicting values → choose most frequent (mode)
                merged_row[col] = group[col].mode()[0]
        
        merged_records.append(merged_row)
    
    return pd.DataFrame(merged_records)


In [ ]:
def clean_customer_dataset(df, original_customer_dataset_name):
    """
    Main cleaning pipeline for customer dataset.
    Executes all stages in proper order:
    0. Column Normalization
    1. Schema & Column Validation
    2. Duplicate Entry Removal
    3. Standardization & Normalization
    4. Missing Value Handling
    5. Outlier Detection
    6. Deduplication
    Finally, saves the cleaned dataset and returns it.
    """
    print("🚀 Starting data cleaning pipeline...\n")

    # =============================================
    # STAGE 0: NORMALIZE COLUMN NAMES
    # =============================================
    print("========== [STAGE 0 START] Normalize Column Names ==========")
    df = normalize_columns_name(df)
    print("✅ [STAGE 0 COMPLETE] Column names normalized.\n")

    # =============================================
    # STAGE 1: SCHEMA & COLUMN VALIDATION
    # =============================================
    print("========== [STAGE 1 START] Schema & Column Validation ==========")
    df, optional_msg = check_optional_columns(df)
    df, mandatory_msg = check_mandatory_columns(df)
    print(optional_msg)
    print(mandatory_msg)
    print("✅ [STAGE 1 COMPLETE] Schema validation done.\n")

    # =============================================
    # STAGE 2: REMOVE DUPLICATE ENTRY ROWS
    # =============================================
    print("========== [STAGE 2 START] Remove Duplicate Entry Rows ==========")
    df = remove_duplicate_entries(df)
    print("✅ [STAGE 2 COMPLETE] Duplicate entries removed.\n")

    # =============================================
    # STAGE 3: STANDARDIZATION & NORMALIZATION
    # =============================================
    print("========== [STAGE 3 START] Standardization & Normalization ==========")
    df = standardize_customer_id(df)
    df = standardize_dob(df)
    df = derive_age_features(df)
    df = derive_age_group(df)
    df = drop_dob_after_age_derived(df)
    df = standardize_gender(df)
    df = standardize_location(df)
    print("✅ [STAGE 3 COMPLETE] Standardization and normalization finished.\n")

    # =============================================
    # STAGE 4: MISSING VALUE HANDLING
    # =============================================
    print("========== [STAGE 4 START] Missing Value Handling ==========")
    df = handle_missing_values(df)
    print("✅ [STAGE 4 COMPLETE] Missing values handled.\n")

    # =============================================
    # STAGE 5: OUTLIER DETECTION
    # =============================================
    print("========== [STAGE 5 START] Outlier Detection ==========")
    df = detect_outliers(df)   # make sure detect_outliers returns df
    print("✅ [STAGE 5 COMPLETE] Outliers handled.\n")

    # =============================================
    # STAGE 6: DEDUPLICATION
    # =============================================
    print("========== [STAGE 6 START] Deduplication ==========")
    df = deduplicate_customers(df)
    print("✅ [STAGE 6 COMPLETE] Duplicate CustomerIDs deduplicated.\n")

    # =============================================
    # SAVE CLEANED DATASET
    # =============================================
    print("========== [FINAL STAGE START] Save Cleaned Dataset ==========")
    base_name, ext = os.path.splitext(original_customer_dataset_name)
    cleaned_file = f"{base_name}_cleaned{ext}"
    df.to_csv(cleaned_file, index=False)
    print(f"✅ [FINAL STAGE COMPLETE] Cleaned dataset saved as: {cleaned_file}\n")

    print("==========================================================")
    print("🎉 Data cleaning pipeline completed successfully!\n")
    return df, cleaned_file


In [8]:
# Keep a copy for comparison
original_customer_df = customer_df.copy()
cleaned_customer_df, cleaned_file_name = clean_customer_dataset(customer_df, original_customer_dataset_name)

def generate_before_after_summary(df_before, df_after, example_rows=1):
    """
    Generate a summary table comparing before and after cleaning:
    - Column name
    - Missing before
    - Missing after
    - Number of invalid/changed entries
    - Example of correction (first few corrected entries)
    """
    summary_data = []
    for col in df_before.columns:
        before_missing = df_before[col].isna().sum()
        after_missing = df_after[col].isna().sum()
        
        # Count changed or corrected entries
        if col in df_after.columns:
            invalid_count = (df_before[col] != df_after[col]).sum()
            # Get example corrections
            example_corrections = df_after.loc[df_before[col] != df_after[col], col].head(example_rows).tolist()
            example_str = ", ".join([str(x) for x in example_corrections]) if example_corrections else "-"
        else:
            invalid_count = "-"
            example_str = "-"
        
        summary_data.append([col, before_missing, after_missing, invalid_count, example_str])
    
    summary_df = pd.DataFrame(summary_data, columns=[
        "Column", "Before Missing", "After Missing", "Invalid/Changed Count", "Example of Correction"
    ])
    
    return summary_df

# Generate table
before_after_summary = generate_before_after_summary(original_customer_df, cleaned_customer_df)

# Display nicely
print("\n===== Before vs After Cleaning Summary =====\n")
print(before_after_summary)



🚀 Starting data cleaning pipeline...

========== STAGE 1: SCHEMA & COLUMN VALIDATION ==========
We noticed that very few entries were provided for date of birth, gender. These columns have been removed. Segmentation will still be performed using geographic (City, State, Country) and behavioral data (e.g., orders, purchase items, total spend).
All mandatory columns have sufficient data and are ready for cleaning.

Missing Data Summary:
customerid: 0.0% missing
city: 44.8% missing
state: 46.9% missing
country: 44.7% missing

========== STAGE 2: REMOVE DUPLICATE ENTRY ROW ==========
Duplicate entries removed.

========== STAGE 3: STANDARDIZATION & NORMALIZATION ==========


C:\Users\user\AppData\Local\Temp\ipykernel_11504\2075746767.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['customerid'] = df['customerid'].astype(str).str.strip().str.upper()


Standardization and normalization completed.

========== STAGE 4: MISSING VALUE HANDLING ==========
Missing values handled successfully.

========== STAGE 5: OUTLIER DETECTION ==========
Outliers handled (e.g., unrealistic ages set to NaN).

========== STAGE 6: DEDUPLICATION ==========


C:\Users\user\AppData\Local\Temp\ipykernel_11504\2075746767.py:197: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.loc[df.groupby('customerid').apply(lambda x: x.notna().sum(axis=1).idxmax())]


Duplicate CustomerIDs deduplicated by data completeness.

✅ Data cleaning pipeline completed successfully!
Cleaned dataset saved as: 2021 - 2025 Customer - Copy_cleaned.csv
Complete data cleaning pipeline execution for customer dataset.


KeyError: 'CustomerID'

In [9]:
malaysia_states = pycountry.subdivisions.get(country_code='MY')

# Print them
for state in malaysia_states:
    print(f"{state.code} - {state.name}")

MY-09 - Perlis
MY-02 - Kedah
MY-16 - Wilayah Persekutuan Putrajaya
MY-13 - Sarawak
MY-06 - Pahang
MY-10 - Selangor
MY-03 - Kelantan
MY-14 - Wilayah Persekutuan Kuala Lumpur
MY-07 - Pulau Pinang
MY-11 - Terengganu
MY-04 - Melaka
MY-15 - Wilayah Persekutuan Labuan
MY-08 - Perak
MY-01 - Johor
MY-12 - Sabah
MY-05 - Negeri Sembilan
